In [4]:
pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
import gc
import textstat

file_path = '/content/drive/My Drive/266 Project/efcamdat_sub.csv'

df = pd.read_csv(file_path)

print(df.head())

       id  level  unit  learner_id learner_nationality  grade  \
0  679604      7     3      114335                  br     90   
1  151196      9     2      136139                  sa     94   
2  117084      9     4       34715                  br     88   
3  113857      7     6       90269                  fr     90   
4   22083      9     3       48465                  br     94   

                      date  topic_id  \
0  2013-11-09 20:50:31.707        51   
1  2012-09-25 06:01:08.117        66   
2  2011-08-28 08:01:15.677        68   
3  2011-07-31 14:51:22.547        54   
4  2011-08-31 16:41:04.210        67   

                                                text  cefr_numeric  \
0  From:l AS xxx@hotmail.com To: AS xxx@IXW.corpo...             3   
1  I am so glad to receive this email from you. A...             3   
2  Hi Fun Skydive, so I give up of my idea. I und...             3   
3  Dear James, Some serious problems have been br...             3   
4  Dear Sue, Thank

**Check data**

In [6]:
df.columns

Index(['id', 'level', 'unit', 'learner_id', 'learner_nationality', 'grade',
       'date', 'topic_id', 'text', 'cefr_numeric', 'cefr_grouped'],
      dtype='object')

In [7]:
df.shape[0]

377967

In [8]:
# Check unique label values
df['labels'] = df['cefr_numeric'].apply(lambda x: x-1)

In [9]:
# Filter out invalid labels
df = df[df['labels'].isin(range(6))]

In [10]:
# Check unique label values after filtering
print("Unique label values after filtering:", df['labels'].unique())

Unique label values after filtering: [2 1 0 3 4 5]


In [11]:
text = df['text']
labels = df['labels']

In [12]:
# Validation and test set sizes
test_size = int(0.1 * len(df))  # 10% for testing
valid_size = int(0.2 * len(df))  # 20% for validation

In [13]:
# Split off the test set
train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=test_size, shuffle=True, random_state=42)

# Split off the validation set from the remaining training data
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=valid_size, shuffle=True, random_state=42)

In [14]:
len(test_texts)

37796

In [15]:
# Unique label values in training set
print("Unique label values in training set:", train_labels.unique())
print("Unique label values in validation set:", valid_labels.unique())
print("Unique label values in test set:", test_labels.unique())

Unique label values in training set: [2 0 1 3 4 5]
Unique label values in validation set: [2 0 3 1 4 5]
Unique label values in test set: [5 1 2 3 0 4]


**Divide the data into training, validation, and test sets**

In [18]:
# Reduce the size of the training and validation sets for faster processing
train_texts = train_texts[:37796]
train_labels = train_labels[:37796]
valid_texts = valid_texts[:37796]
valid_labels = valid_labels[:37796]
test_texts = test_texts[:37796]
test_labels = test_labels[:37796]

In [19]:
# Extract ARI readability scores
def extract_ari_features(texts):
    ari_scores = [textstat.automated_readability_index(text) for text in texts]
    return ari_scores


In [20]:
train_ari_scores = extract_ari_features(train_texts)
valid_ari_scores = extract_ari_features(valid_texts)
test_ari_scores = extract_ari_features(test_texts)

In [21]:
# Tokenize the data
max_length = 50
rtokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [22]:
def tokenize_texts(texts, tokenizer, max_length):
    if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
        raise ValueError("Input texts should be a list of strings.")
    return tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

In [23]:
def safe_tokenize(texts, tokenizer, max_length):
    try:
        encodings = tokenize_texts(texts, tokenizer, max_length)
        if 'input_ids' not in encodings:
            raise ValueError("Tokenization did not produce 'input_ids'.")
        print(f"Successfully tokenized {len(texts)} texts.")
        return encodings
    except ValueError as e:
        print(f"Tokenization error: {e}")
        return None


In [24]:
train_encodings = safe_tokenize(list(train_texts), rtokenizer, max_length)
valid_encodings = safe_tokenize(list(valid_texts), rtokenizer, max_length)
test_encodings = safe_tokenize(list(test_texts), rtokenizer, max_length)

Successfully tokenized 37796 texts.
Successfully tokenized 37796 texts.
Successfully tokenized 37796 texts.


In [25]:
if train_encodings is None or valid_encodings is None or test_encodings is None:
    raise ValueError("Tokenization failed for one or more datasets.")

# Print the keys of the tokenized encodings to verify
print(f"Keys of train_encodings: {train_encodings.keys()}")
print(f"Keys of valid_encodings: {valid_encodings.keys()}")
print(f"Keys of test_encodings: {test_encodings.keys()}")

Keys of train_encodings: dict_keys(['input_ids', 'attention_mask'])
Keys of valid_encodings: dict_keys(['input_ids', 'attention_mask'])
Keys of test_encodings: dict_keys(['input_ids', 'attention_mask'])


In [26]:
# Convert to TensorFlow Datasets with ARI scores
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids_layer": train_encodings.input_ids, "attention_mask_layer": train_encodings.attention_mask, "ari_score": train_ari_scores},
    train_labels
)).shuffle(len(train_texts)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids_layer": valid_encodings.input_ids, "attention_mask_layer": valid_encodings.attention_mask, "ari_score": valid_ari_scores},
    valid_labels
)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids_layer": test_encodings.input_ids, "attention_mask_layer": test_encodings.attention_mask, "ari_score": test_ari_scores},
    test_labels
)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)

In [27]:
# Check TensorFlow Dataset Elements
for batch in test_dataset.take(1):
    inputs, labels = batch
    print(f"Input IDs shape: {inputs['input_ids_layer'].shape}")
    print(f"Attention Mask shape: {inputs['attention_mask_layer'].shape}")
    print(f"ARI Score shape: {inputs['ari_score'].shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Label values: {labels.numpy()}")

Input IDs shape: (16, 50)
Attention Mask shape: (16, 50)
ARI Score shape: (16,)
Labels shape: (16,)
Label values: [5 1 1 2 3 2 1 0 0 3 0 0 4 2 2 2]


**Model architecture**

In [28]:
# Define the model
def create_roberta_cl_model(model, num_classes=6, dropout=0.3, learning_rate=0.0001):
    model.trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')
    ari_score = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='ari_score')

    model_inputs = [input_ids, attention_mask, ari_score]
    model_out = model(input_ids=input_ids, attention_mask=attention_mask)
    model_out = model_out.last_hidden_state

    conv = tf.keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu')(model_out)
    conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
    conv = tf.keras.layers.Dropout(dropout)(conv)
    lstm = tf.keras.layers.LSTM(units=256, return_sequences=False, return_state=False)(conv)
    lstm = tf.keras.layers.Dropout(dropout)(lstm)

    # Concatenate LSTM output with the ARI score
    concatenated = tf.keras.layers.Concatenate()([lstm, ari_score])

    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(concatenated)

    classification_model = tf.keras.Model(inputs=model_inputs, outputs=classification)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=0.1)
    optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

    classification_model.compile(optimizer=optimizer,
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 metrics=['accuracy'])

    return classification_model

In [29]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')
roberta_cl_model = create_roberta_cl_model(model=roberta_model, num_classes=6)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [30]:
# Clear session and free memory before starting the training
tf.keras.backend.clear_session()
gc.collect()

25908

In [31]:
# Display model summary
roberta_cl_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids_layer (InputLaye  [(None, 50)]                 0         []                            
 r)                                                                                               
                                                                                                  
 attention_mask_layer (Inpu  [(None, 50)]                 0         []                            
 tLayer)                                                                                          
                                                                                                  
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPooli   1246456   ['input_ids_layer[0][0]',     
 aModel)                     ngAndCrossAttentions(last_   32         'attention_mask_layer[0][

In [32]:
# dataset = tf.data.Dataset.from_tensor_slices(({"input_ids_layer": train_encodings.input_ids, "attention_mask_layer": train_encodings.attention_mask}, train_labels))
# dataset = dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

vdataset = tf.data.Dataset.from_tensor_slices(({"input_ids_layer": valid_encodings.input_ids, "attention_mask_layer": valid_encodings.attention_mask}, valid_labels))
vdataset = vdataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [33]:

# Train the model
roberta_cl_model_history = roberta_cl_model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=1
)

2363/2363 [==============================] - 15108s 6s/step - loss: 0.7313 - accuracy: 0.7267 - val_loss: 0.5703 - val_accuracy: 0.7980


In [34]:
# Predict and evaluate the model
preds = roberta_cl_model.predict(test_dataset)
pred_labels = tf.argmax(preds, axis=-1)
print(classification_report(test_labels, pred_labels))

2363/2363 [==============================] - 7115s 3s/step
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      9970
           1       0.91      0.74      0.82      9918
           2       0.67      0.95      0.79     10079
           3       0.87      0.62      0.73      6184
           4       0.49      0.57      0.53      1444
           5       0.00      0.00      0.00       201

    accuracy                           0.81     37796
   macro avg       0.65      0.63      0.63     37796
weighted avg       0.83      0.81      0.81     37796



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
